### Versi 2

In [1]:
# ===========================================
# Cell 1. Import Library dan Setup Environment
# ===========================================

import os
import json
import random
from pathlib import Path
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import zipfile
import io

# Gunakan GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device yang digunakan:", device)

Device yang digunakan: cuda


In [2]:
# ===========================================
# Cell 2. Fungsi Bantuan Umum
# ===========================================

from matplotlib.colors import ListedColormap

def seed_everything(seed=42):
    """Menetapkan seed random agar hasil eksperimen bisa direplikasi"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(42)

def visualize_tile(x_tile, y_true=None, y_pred=None, json_path=None,class_names=None, idx=0):
    """
    Menampilkan citra tile beserta mask ground-truth dan prediksi
    """
    if isinstance(x_tile, torch.Tensor):
        x = x_tile.cpu().numpy()
        x = np.transpose(x, (1,2,0))  # ubah dari [B,H,W] -> [H,W,B]
    else:
        x = x_tile

    # menampilkan pseudo-RGB (karena data hyperspectral)
    B = x.shape[2]
    b1, b2, b3 = int(B*0.05), int(B*0.5), int(B*0.9)
    rgb = x[..., [b1, b2, b3]]
    rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-9)

    # Coba baca colormap dari file JSON
    if json_path and os.path.exists(json_path):
        with open(json_path, "r") as f:
            label_info = json.load(f)
        custom_colors = [c["color"][:7] for c in label_info]
        cmap = ListedColormap(custom_colors)
    else:
        print("File json tidak terbaca, menggunakan cmap tab20")
        cmap = "tab20"  # fallback

        

    # Visualisasi
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(rgb_norm); plt.title("Citra (Pseudo-RGB)")
    if y_true is not None:
        plt.subplot(1,3,2); plt.imshow(y_true, cmap=cmap); plt.title("Ground Truth")
    if y_pred is not None:
        plt.subplot(1,3,3); plt.imshow(y_pred, cmap=cmap); plt.title("Prediksi")
    plt.show()



In [9]:
# =========================================================
# Cell 3. Dataset Loader (SeaweedDataset) dan Label Mapping
# =========================================================

def load_label_mapping(json_path):
    """Membaca file label_classes.json untuk mapping id ke nama kelas"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    idx_to_name = {i: item["name"] for i, item in enumerate(data)}
    return idx_to_name

# def normalize_reflectance(cube):
#     """Menormalkan nilai reflektansi ke rentang 0-1 per tile"""
#     cube = np.nan_to_num(cube).astype(np.float32)
#     min_val = np.nanmin(cube)
#     max_val = np.nanmax(cube)
#     if max_val > min_val:
#         cube = (cube - min_val) / (max_val - min_val)
#     return cube

def normalize_reflectance(cube):
    """Menormalkan reflektansi 0–1 per tile, hemat RAM, aman untuk mmap read-only."""
    # Jika hasil np.load mmap, array biasanya read-only → buat copy ringan
    if not cube.flags.writeable:
        cube = cube.astype(np.float32, copy=True)  # hanya salin tile, bukan file besar

    # Pastikan float32
    if cube.dtype != np.float32:
        cube = cube.astype(np.float32, copy=False)

    # Ganti NaN / Inf in-place
    np.nan_to_num(cube, copy=False)

    # Normalisasi min-max
    min_val = np.nanmin(cube)
    max_val = np.nanmax(cube)
    if max_val > min_val:
        cube -= min_val
        cube /= (max_val - min_val + 1e-8)

    return cube




class SeaweedDataset(Dataset):
    """
    Dataset hemat memori berbasis file .npy hasil konversi.
    Membaca tile langsung dari disk menggunakan mmap_mode="r".
    """
    def __init__(self, data_files, label_map, tile_size=128, normalize=True, label_remap=None):
        self.data_files = data_files
        self.label_map = label_map
        self.tile_size = tile_size
        self.normalize = normalize
        # label_remap: dict {orig_label: new_index}, if None -> identity mapping
        self.label_remap = label_remap

        # Daftar pasangan (file_x, file_y)
        self.pairs = []
        for f in data_files:
            if f.endswith("_x.npy"):
                fy = f.replace("_x.npy", "_y.npy")
                if os.path.exists(fy):
                    self.pairs.append((f, fy))
        
        # Hanya menyimpan indeks tile berdasarkan ukuran file .npy
        self.index = []  
        for file_idx, (fx, fy) in enumerate(self.pairs):
            x = np.load(fx, mmap_mode="r")
            H, W, _ = x.shape
            for i in range(0, H - tile_size + 1, tile_size):
                for j in range(0, W - tile_size + 1, tile_size):
                    self.index.append((file_idx, i, j))
            del x  # bebaskan referensi memori

        print(f"[INFO] Total tile terdaftar: {len(self.index)} dari {len(self.pairs)} file")

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        file_idx, i, j = self.index[idx]
        fx, fy = self.pairs[file_idx]
        
        # Memuat tile menggunakan mmap
        x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
        y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]

        # Abaikan tile kosong (semua 0) dengan fail-safe agar tidak infinite recursion
        # kalau tetap kosong setelah 3 percobaan → biarkan saja y tetap kosong (model akan skip naturally karena weight=0 utk bg)
        for _ in range(3):  # coba maksimal 3 kali
            if np.any(y > 0):
                break
            file_idx, i, j = self.index[np.random.randint(0, len(self.index))]
            fx, fy = self.pairs[file_idx]
            x = np.load(fx, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size, :]
            y = np.load(fy, mmap_mode="r")[i:i+self.tile_size, j:j+self.tile_size]


        if self.normalize:
            x = normalize_reflectance(x)

        # REMAP label bila mapping diberikan
        if self.label_remap is not None:
            # buat array output dengan nilai default 0 (background) atau -1 jika ingin ignore
            y_remap = np.zeros_like(y, dtype=np.int64)
            # set default to 0 (background) then map others
            for orig_label, new_idx in self.label_remap.items():
                # gunakan boolean mask assignment (efisien)
                if orig_label == 0:
                    # background -> keep 0 (or explicitly assign)
                    y_remap[y == orig_label] = new_idx
                else:
                    y_remap[y == orig_label] = new_idx
            y = y_remap
        else:
            y = y.astype(np.int64)

        # Konversi ke tensor
        x_tensor = torch.tensor(x.transpose(2, 0, 1), dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        return x_tensor, y_tensor


def detect_actual_classes(pairs):
    """Scan semua file y.npy untuk mendeteksi kelas yang benar-benar ada"""
    found = set()
    for _, fy in pairs:
        y = np.load(fy, mmap_mode="r")
        found |= set(np.unique(y))
    found = sorted(list(found))
    print(f"[INFO] Kelas AKTUAL yang ditemukan di dataset: {found}")
    return found

In [10]:
# ==============================================
# Cell 4. Load Dataset dan FILE-LEVEL Splitting
# ==============================================

data_dir = "../data/npy_converted"
label_json_path = "../data/annotation/segmentation_masks/label_classes.json"

label_map = load_label_mapping(label_json_path)
print(f"Jumlah total kelas di JSON: {len(label_map)}")

# Ambil semua file _x.npy
all_x_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith("_x.npy")])
pairs = [(fx, fx.replace("_x.npy", "_y.npy")) for fx in all_x_files if os.path.exists(fx.replace("_x.npy", "_y.npy"))]

print(f"Total pasangan file X-Y ditemukan: {len(pairs)} (expected: 18)")
for p in pairs[:5]:
    print("Contoh:", os.path.basename(p[0]), "<->", os.path.basename(p[1]))

# Split deterministik berbasis urutan nama (11 train, 5 val, 3 test)
train_pairs = pairs[:11]
val_pairs   = pairs[11:16]
test_pairs  = pairs[16:]

print("\n=== FINAL SPLIT PER FILE ===")
print(f"Train : {len(train_pairs)}")
print(f"Val   : {len(val_pairs)}")
print(f"Test  : {len(test_pairs)}")

# DETEKSI kelas aktual yang benar-benar muncul (bukan ambil dari JSON)
actual_classes = detect_actual_classes(train_pairs + val_pairs + test_pairs)

# actual_classes adalah list of numpy ints e.g. [0,8,12,13,14,18,38]
orig_classes = [int(x) for x in actual_classes]  # cast to python ints
# Buat mapping orig_label -> contiguous idx 0..(C-1)
label_remap = {orig: idx for idx, orig in enumerate(orig_classes)}
print(f"[INFO] Label remap (orig -> new): {label_remap}")

# Update datasets: pass label_remap ke SeaweedDataset
train_dataset = SeaweedDataset([p[0] for p in train_pairs], label_map, tile_size=128, label_remap=label_remap)
val_dataset   = SeaweedDataset([p[0] for p in val_pairs], label_map, tile_size=128, label_remap=label_remap)
test_dataset  = SeaweedDataset([p[0] for p in test_pairs], label_map, tile_size=128, label_remap=label_remap, normalize=False)


# Hitung frekuensi kelas dari TRAIN dataset (hitung dari file y, lebih efisien)
'''
Menghitung distribusi jumlah pixel dari setiap kelas di TRAIN SET
Lalu menghitung bobot loss yang adil (class weights) berdasarkan distribusi ini
Supaya kelas langka tidak tertindas / diabaikan oleh model, 
karena dataset Imbalance besar (background 0 paling dominan)
'''
from collections import Counter
counter = Counter()
for _, fy in train_pairs:
    y = np.load(fy, mmap_mode="r")
    # remap using label_remap quickly:
    for orig, new in label_remap.items():
        cnt = int((y == orig).sum())
        counter[new] += cnt

print(f"[INFO] Pixel counts per (remapped) class (train set): {dict(counter)}")

# Buat class weights (inverse frequency), dan set weight[0]=0 karena ignore_index=0
counts = np.array([counter.get(i, 0) for i in range(len(label_remap))], dtype=np.float64)
eps = 1e-6
inv_freq = 1.0 / (counts + eps)
# optional normalization so that mean weight = 1
inv_freq = inv_freq / np.mean(inv_freq)
# set background index weight to 0 (ignored)
inv_freq[0] = 0.0

print(f"[INFO] Class weights (remapped, sum-normalized, bg=0): {inv_freq}")

# Simpan nilai-nilai penting ke variabel global untuk digunakan Cell6
num_classes_actual = len(label_remap)
label_remap_global = label_remap
class_weights_np = inv_freq.astype(np.float32)

print(f"\nTotal TILE train: {len(train_dataset)}, val: {len(val_dataset)}, test: {len(test_dataset)}")


Jumlah total kelas di JSON: 41
Total pasangan file X-Y ditemukan: 18 (expected: 18)
Contoh: massimal_smola_maholmen_202306211129-2_hsi_003_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_003_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_004_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_004_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_008_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_008_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_009_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_009_processed_y.npy
Contoh: massimal_smola_maholmen_202306211129-2_hsi_013_processed_x.npy <-> massimal_smola_maholmen_202306211129-2_hsi_013_processed_y.npy

=== FINAL SPLIT PER FILE ===
Train : 11
Val   : 5
Test  : 2
[INFO] Kelas AKTUAL yang ditemukan di dataset: [np.int32(0), np.int32(8), np.int32(12), np.int32(13), np.int32(14), np.int32(18), np.int32(38)]
[INFO] Label remap (o

In [14]:
# === CELL DEBUG (BUKAN CELL URUTAN ===

# print(np.load(train_pairs[0][0], mmap_mode="r").shape)
# print(np.load(train_pairs[1][0], mmap_mode="r").shape)
# print(np.load(train_pairs[-1][0], mmap_mode="r").shape)

import torch
print("CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available: True
Current device: NVIDIA GeForce RTX 4080 SUPER


In [11]:
# ======================================================
# Cell 5. Model Fully Convolutional HybridSN (3D+2D CNN)
# ======================================================

class FCHybridSN(nn.Module):
    def __init__(self, in_bands=300, num_classes=7): # nilai num_classes dijadikan default 7
        super().__init__()
        self.conv3d_1 = nn.Conv3d(1, 16, (7,3,3), padding=(0,1,1))
        self.bn3d_1 = nn.BatchNorm3d(16)
        self.conv3d_2 = nn.Conv3d(16, 32, (5,3,3), padding=(0,1,1))
        self.bn3d_2 = nn.BatchNorm3d(32)
        self.conv3d_3 = nn.Conv3d(32, 64, (3,3,3), padding=(0,1,1))
        self.bn3d_3 = nn.BatchNorm3d(64)

        self._out_spec = in_bands - 12
        mid_ch = 256
        self.conv2d_1 = nn.Conv2d(64 * max(1, self._out_spec), mid_ch, 3, padding=1)
        self.bn2d_1 = nn.BatchNorm2d(mid_ch)
        self.conv2d_2 = nn.Conv2d(mid_ch, 128, 3, padding=1)
        self.bn2d_2 = nn.BatchNorm2d(128)
        self.conv2d_3 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn2d_3 = nn.BatchNorm2d(64)
        self.classifier = nn.Conv2d(64, num_classes, 1)

    def forward(self, x):
        B, Bands, H, W = x.shape
        x3 = x.unsqueeze(1)
        x3 = F.relu(self.bn3d_1(self.conv3d_1(x3)))
        x3 = F.relu(self.bn3d_2(self.conv3d_2(x3)))
        x3 = F.relu(self.bn3d_3(self.conv3d_3(x3)))
        B, C3, out_spec, H, W = x3.shape
        x2 = x3.view(B, C3 * out_spec, H, W)
        x2 = F.relu(self.bn2d_1(self.conv2d_1(x2)))
        x2 = F.relu(self.bn2d_2(self.conv2d_2(x2)))
        x2 = F.relu(self.bn2d_3(self.conv2d_3(x2)))
        return self.classifier(x2)

# Ambil jumlah band langsung dari data TRAIN pertama
sample_x = np.load(train_pairs[0][0], mmap_mode="r")
in_bands_actual = sample_x.shape[2]  # ambil jumlah band asli dari npy

print(f"Band input aktual terdeteksi: {in_bands_actual}")

# GUNAKAN jumlah kelas AKTUAL (bukan 41 dari JSON!)
model = FCHybridSN(in_bands=in_bands_actual, num_classes=num_classes_actual).to(device)

print(model)


Band input aktual terdeteksi: 300
FCHybridSN(
  (conv3d_1): Conv3d(1, 16, kernel_size=(7, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_2): Conv3d(16, 32, kernel_size=(5, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (bn3d_3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_1): Conv2d(18432, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2d_1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_2): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2d_2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1,

In [12]:
# ===========================================
# Cell 6. Keperluan Evaluasi & Loss / Optimizer (diperbarui)
# ===========================================
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Menggunakan class weights yang sudah dihitung
# pastikan untuk memindahkan weight ke device nanti saat membangun criterion (lakukan setelah model.to(device))
weight_tensor = torch.from_numpy(class_weights_np)

# enable cudnn benchmark untuk kecepatan (bagus ketika input sizes konstan)
torch.backends.cudnn.benchmark = True

# hyperparameters (adjustable)
LR = 1e-4
WEIGHT_DECAY = 1e-5
BATCH_SIZE = 1   # Jika OOM, turunkan ke 2 atau 1. Coba 2 jika masih OOM.

# DataLoaders menggunakan pin_memory dan num_workers untuk GPU
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
# val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
# test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# ==== Dataloaders alternatif dengan mematikan multiprocessing
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)


# model to device sudah dilakukan lebih awal; pastikan criterion menggunakan weights on device
criterion = nn.CrossEntropyLoss(weight=None, ignore_index=0)  # placeholder weight = 0, akan di-set setelah move model to device

# Optimizer & scheduler (setelah model available)
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

# Mixed precision scaler
# scaler = torch.cuda.amp.GradScaler(enabled=(device.type == 'cuda')) # penggunaan "enabled=" sudah deprecated, ganti dengan yang bawah
scaler = torch.amp.GradScaler(device.type)

def pixel_accuracy(pred, target):
    valid = (target >= 0)
    correct = (pred[valid] == target[valid]).sum()
    total = valid.sum()
    return (correct.float() / (total.float() + 1e-9)).item()

def iou_per_class(pred, target, num_classes):
    ious = []
    for cls in range(num_classes):
        pred_i = (pred == cls)
        target_i = (target == cls)
        inter = (pred_i & target_i).sum()
        union = (pred_i | target_i).sum()
        if union == 0:
            ious.append(float('nan'))
        else:
            ious.append((inter.float() / union.float()).item())
    return ious


In [13]:
# ===========================================
# Cell 7. Loop Training Utama (+ checkpoint) — AMP & class weights
# ===========================================
import os, time
from tqdm import tqdm

START_EPOCH = 1     # mulai default 1 (kamu sebelumnya sempat pakai 10)
NUM_EPOCHS = 50     # boleh disesuaikan; start conservative
best_val_acc = 0.0

checkpoint_path = "hybridsn_sgmt_ver2_checkpoint.pth"

# Pastikan criterion menggunakan weights pada device
weight_tensor_device = torch.from_numpy(class_weights_np).to(device)
criterion = nn.CrossEntropyLoss(weight=weight_tensor_device, ignore_index=0)

# Load checkpoint jika ada (including scaler & scheduler)
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    scaler.load_state_dict(checkpoint.get("scaler_state", {}))
    START_EPOCH = checkpoint["epoch"] + 1
    best_val_acc = checkpoint.get("best_val_acc", 0.0)
    print(f"[INFO] Memuat checkpoint {checkpoint_path}, resume epoch {START_EPOCH}")
else:
    print("[INFO] Tidak ditemukan checkpoint. Mulai training dari awal.")

for epoch in range(START_EPOCH, NUM_EPOCHS + 1):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}", leave=True)
    for xb, yb in pbar:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            logits = model(xb)               # [B, C, H, W]
            loss = criterion(logits, yb)

        # scaler backward + step
        if device.type == 'cuda':
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({"TrainLoss": f"{loss.item():.4f}"})

    avg_loss = running_loss / max(1, len(train_loader))

    # Validation
    model.eval()
    val_accs = []
    val_ious = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
                logits = model(xb)
            preds = logits.argmax(dim=1)
            val_accs.append(pixel_accuracy(preds, yb))
            val_ious.extend(iou_per_class(preds, yb, num_classes_actual))

    mean_val_acc = np.nanmean(val_accs)
    mean_iou = np.nanmean([v for v in val_ious if not np.isnan(v)])
    elapsed = time.time() - start_time

    print(f"Epoch {epoch}/{NUM_EPOCHS} | Loss={avg_loss:.4f} | ValAcc={mean_val_acc:.4f} | mIoU={mean_iou:.4f} | Time={elapsed/3600:.2f} jam")

    # Scheduler step (ReduceLROnPlateau uses validation metric)
    scheduler.step(mean_val_acc)

    # Checkpoint (simpan juga scaler state)
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_val_acc": best_val_acc,
        "scaler_state": scaler.state_dict() if device.type == 'cuda' else {}
    }
    torch.save(checkpoint, checkpoint_path)

    if mean_val_acc > best_val_acc:
        best_val_acc = mean_val_acc
        torch.save(checkpoint, "hybridsn_sgmt_ver2_best.pth")
        print("[INFO] Model terbaik disimpan.")


[INFO] Tidak ditemukan checkpoint. Mulai training dari awal.


Epoch 1/50:   0%|                                                                             | 0/1099 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_13764\2700568697.py:39: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
Epoch 1/50: 100%|████████████████████████████████████████████████| 1099/1099 [04:09<00:00,  4.40it/s, TrainLoss=1.0853]
C:\Users\admin\AppData\Local\Temp\ipykernel_13764\2700568697.py:65: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Epoch 1/50 | Loss=nan | ValAcc=0.2537 | mIoU=0.0542 | Time=0.09 jam
[INFO] Model terbaik disimpan.


Epoch 2/50: 100%|████████████████████████████████████████████████| 1099/1099 [04:08<00:00,  4.43it/s, TrainLoss=1.3037]


Epoch 2/50 | Loss=nan | ValAcc=0.1669 | mIoU=0.0369 | Time=0.09 jam


...

Epoch 48/50 | Loss=nan | ValAcc=0.1153 | mIoU=0.0314 | Time=0.09 jam


Epoch 49/50: 100%|███████████████████████████████████████████████| 1099/1099 [04:14<00:00,  4.32it/s, TrainLoss=0.0639]


Epoch 49/50 | Loss=nan | ValAcc=0.2246 | mIoU=0.0587 | Time=0.09 jam


Epoch 50/50: 100%|███████████████████████████████████████████████| 1099/1099 [04:13<00:00,  4.34it/s, TrainLoss=0.2190]


Epoch 50/50 | Loss=nan | ValAcc=0.2554 | mIoU=0.0774 | Time=0.09 jam


In [15]:
print(class_weights_np)

[0.         0.2185292  0.11722793 0.22719245 4.9649825  1.3176336
 0.14149204]
